In [11]:
import mne
import numpy as np
import utils
from sklearn.decomposition import PCA
from mne_icalabel import label_components
from matplotlib import pyplot as plt
from matplotlib.widgets import Slider
from pyprep.prep_pipeline import PrepPipeline
from scipy import signal
from mne.preprocessing import ICA
import re
import os

In [7]:
data_raw =utils.load_and_dump_channels(r"A:\Sano\adult_data\chb16\chb16_17.edf")
utils.reorder_channels(data_raw)
ica, ic_labels, raw_copy = utils.run_preprocessing(data_raw,18)

C:\Users\Szymon Mazurek\AppData\Local\Temp\ipykernel_22156\1842315277.py:2: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8', '-'}. Applying running numbers for duplicates.
  data_raw = mne.io.read_raw_edf(filepath,preload = False,verbose=False)
C:\Users\Szymon Mazurek\AppData\Local\Temp\ipykernel_22156\1842315277.py:2: RuntimeWarning: Scaling factor is not defined in following channels:
--0, --1, --2, --3, --4
  data_raw = mne.io.read_raw_edf(filepath,preload = False,verbose=False)


Removing channel --0
Removing channel --1
Removing channel --2
Removing channel --3
Removing channel --4
Removing channel T8-P8-1


In [ ]:
def run_prep(raw,line_freq,ransac = False, channel_wise = False):
    sfreq = raw.info['sfreq']
    prep_params = {
        "ref_chs": "eeg",
        "reref_chs": "eeg",
        "line_freqs": np.arange(line_freq, sfreq / 2, line_freq),
    }
    raw.load_data()
    montage = raw.get_montage()
    prep = PrepPipeline(raw, prep_params,montage,ransac=ransac,channel_wise=channel_wise)
    prep_output = prep.fit()
    return prep

In [ ]:
signal_for_pca = raw_copy.copy()

In [ ]:
dat_for_pca = signal_for_pca._data

In [ ]:
pca = PCA(18)
pca.fit(dat_for_pca)

In [ ]:
pca = PCA(0.95)
transformed_signal = pca.fit_transform(dat_for_pca)

In [ ]:
signal_for_pca._data = transformed_signal.T

In [ ]:
var = pca.explained_variance_ratio_

In [ ]:
components = pca.components_

In [ ]:
component = components[0]
sampling_f = 256
Plot, Axis = plt.subplots()
secs_per_frame = 5
slider_res = sampling_f*secs_per_frame
# Adjust the bottom size according to the
# requirement of the user
plt.subplots_adjust(bottom=0.25)

slider_color = 'White'
plt.plot(component)
Axis.set_xticks(np.arange(0,component.shape[0]/256))
# Set the axis and slider position in the plot
axis_position = plt.axes([0.2, 0.1, 0.65, 0.03],
                         facecolor = slider_color)
slider_position = Slider(axis_position,
                         'Pos', 0, component.shape[0], valinit=0)
#plt.plot(component)
# update() function to change the graph when the
# slider is in use
comp_y_max = max(component)
comp_y_min = min(component)

def update(val):
    pos = slider_position.val
    Axis.axis([pos, pos+slider_res, comp_y_min, comp_y_max])
    Plot.canvas.draw_idle()
 
# update function called using on_changed() function
slider_position.on_changed(update)
plt.show()

In [ ]:
%matplotlib inline

In [ ]:
fig, axes = plt.subplots(components.shape[0],figsize=(20,40))
sampling_f = 256
mins_to_plot = 2
for n in range(components.shape[0]):
    axes[n].set_title(f"Copmonent {n+1}, explained variance: {round(var[n]*100,3)}%")
    axes[n].plot(components[n][:mins_to_plot*sampling_f])
    #axes[n].set_xticks(np.arange(0,len(components[n]),sampling_f))
fig.tight_layout(pad=1.0)
plt.show()

In [ ]:
fig, axes = plt.subplots(components.shape[0],figsize=(20,40))
sampling_f = 256

for n in range(components.shape[0]):

    freqs, psd = signal.welch(components[n],fs = sampling_f)
    axes[n].set_title(f"Copmonent {n+1}, PSD")
    axes[n].set_xlim(right=45)
    axes[n].plot(freqs,psd)

fig.tight_layout(pad=1.0)
plt.show()

In [ ]:
plt.bar(x= range(1,var.shape[0]+1),height=var*100)


In [ ]:
for n in range(var.shape[0]):
    if sum(var[:n] )>= 0.9:
        n_components = n
        break

In [ ]:
new_data = pca.transform(raw_data)